<a href="https://colab.research.google.com/github/alexferrero23/twitter-category-nlp/blob/develop/src/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import glob
import re
import io

#### Select your file

In [2]:
from google.colab import files
uploaded = files.upload()

Saving working_df (1).orc to working_df (1) (1).orc


In [3]:
import pyarrow.orc 

df = pd.read_orc('working_df (1).orc')

In [4]:
df.head(5)

,username,created_at,text,lang,label,retweet_count,favorite_count,length
0,RepAdams,2022-03-23 18:53:00,"Madeleine Albright often said, “There is a spe...",en,D,56.0,196.0,117
1,RepAdams,2022-03-22 14:40:16,Judge Ketanji Brown Jackson:✔️ Received a unan...,en,D,13.0,26.0,115
2,RepAdams,2022-03-21 20:59:02,When I Grow Up is a virtual forum designed to ...,en,D,5.0,4.0,117
3,RepAdams,2022-03-21 19:55:53,"Ketanji Brown Jackson's parents, Ellery &amp; ...",en,D,8.0,31.0,120
4,RepAdams,2022-03-21 13:58:54,Judge Ketanji Brown Jackson is the most qualif...,en,D,19.0,102.0,117


In [5]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=set(stopwords.words('english'))
import string
list_of_rubish = ['&amp;', '-', '…', '’', '“', '—', '”', 'amp'] #stop_words.extend(("amp"))

def create_corpus_column(tweet):
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    cleaned_corpus = []
    corpus = tokenizer.tokenize(tweet)
    for word in corpus:
        if (word not in stop) and (word not in list_of_rubish) and (word not in string.punctuation):
             cleaned_corpus.append(word)
    return cleaned_corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df['text_corpus'] = df['text'].apply(create_corpus_column)

In [7]:
df.head(5)

,username,created_at,text,lang,label,retweet_count,favorite_count,length,text_corpus
0,RepAdams,2022-03-23 18:53:00,"Madeleine Albright often said, “There is a spe...",en,D,56.0,196.0,117,"[madeleine, albright, often, said, special, pl..."
1,RepAdams,2022-03-22 14:40:16,Judge Ketanji Brown Jackson:✔️ Received a unan...,en,D,13.0,26.0,115,"[judge, ketanji, brown, jackson, ✔, ️, receive..."
2,RepAdams,2022-03-21 20:59:02,When I Grow Up is a virtual forum designed to ...,en,D,5.0,4.0,117,"[grow, virtual, forum, designed, inspire, wome..."
3,RepAdams,2022-03-21 19:55:53,"Ketanji Brown Jackson's parents, Ellery &amp; ...",en,D,8.0,31.0,120,"[ketanji, brown, jackson's, parents, ellery, j..."
4,RepAdams,2022-03-21 13:58:54,Judge Ketanji Brown Jackson is the most qualif...,en,D,19.0,102.0,117,"[judge, ketanji, brown, jackson, qualified, su..."


In [8]:
for i in range(0,20):
  print(df['text_corpus'][i])
  print(df['text'][i])

['madeleine', 'albright', 'often', 'said', 'special', 'place', 'hell', 'women', 'help', 'feminis']
Madeleine Albright often said, “There is a special place in hell for women who don't help each other.” As a feminis… 
['judge', 'ketanji', 'brown', 'jackson', '✔', '️', 'received', 'unanimous', 'well', 'qualified', 'rating', 'nonpartisan', 'american', 'bar', 'ass']
Judge Ketanji Brown Jackson:✔️ Received a unanimous "Well Qualified" rating from the nonpartisan American Bar Ass… 
['grow', 'virtual', 'forum', 'designed', 'inspire', 'women', 'girls', 'open', 'panelists', 'share']
When I Grow Up is a virtual forum designed to inspire women and girls, but is open to all. Our panelists will share… 
['ketanji', 'brown', "jackson's", 'parents', 'ellery', 'johnny', 'brown', 'graduated', 'hbcus', 'giving', 'daughter', 'oppo']
Ketanji Brown Jackson's parents, Ellery &amp; Johnny Brown, both graduated from HBCUs - giving their daughter the oppo… 
['judge', 'ketanji', 'brown', 'jackson', 'qualified', 

In [9]:
# Unnest of text_corpus
df_tidy = df.explode(column='text_corpus')
df_tidy = df_tidy.drop(columns='text')
df_tidy.head(3)

,username,created_at,lang,label,retweet_count,favorite_count,length,text_corpus
0,RepAdams,2022-03-23 18:53:00,en,D,56.0,196.0,117,madeleine
0,RepAdams,2022-03-23 18:53:00,en,D,56.0,196.0,117,albright
0,RepAdams,2022-03-23 18:53:00,en,D,56.0,196.0,117,often


In [10]:
# Data pivot by label
df_pivot = df_tidy.groupby(["label","text_corpus"])["text_corpus"] \
                .agg(["count"]).reset_index() \
                .pivot(index = "text_corpus" , columns="label", values= "count")
df_pivot.columns.name = None

In [11]:
# Consine similarity
from scipy.spatial.distance import cosine

def similitud_coseno(a,b):
    distancia = cosine(a,b)
    return 1-distancia

df_pivot.corr(method=similitud_coseno)

,D,R
D,1.000000,0.918538
R,0.918538,1.000000


#### ***This is actually really bad news, if there is a big correlation between both groups, its gonna be really difficult to say whether a tweet tends to be republican or democrat***

---



#MODEL
#### For the feature extraction, I'm gonna use Tf-idf (Term frequency – Inverse document frequency).

In [12]:
#train-test split
from sklearn.model_selection import train_test_split

datos_X = df.text
datos_y = df.label

X_train, X_test, y_train, y_test = train_test_split(
    datos_X,
    datos_y,
    test_size = 0.2,
    random_state = 123
    
)

In [13]:
#check of proportions
value, counts = np.unique(y_train, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))
value, counts = np.unique(y_test, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))

{'D': 55.51141053977046, 'R': 44.48858946022954}
{'D': 55.40755731898916, 'R': 44.59244268101084}


In [14]:
#Tf-idf matrix
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenize(tweet):
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    corpus = tokenizer.tokenize(tweet)
    return corpus

vectorizer  = TfidfVectorizer(
                        tokenizer  = tokenize,
                        min_df     = 5 #might change
                    )
vectorizer.fit(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(min_df=5, tokenizer=<function tokenize at 0x7ff65cd8e050>)

In [15]:
tfidf_train = vectorizer.transform(X_train)
tfidf_test  = vectorizer.transform(X_test)

In [16]:
print(f" Número de tokens creados: {len(vectorizer.get_feature_names())}")
print(vectorizer.get_feature_names()[50:])

 Número de tokens creados: 47173
['02', '026', '03', '04', '05', '05am', '06', '07', '08', '09', '1', '1,00', '1,000', '1,000+', '1,100', '1,182', '1,200', '1,250', '1,300', '1,400', '1,449', '1,500', '1,536', '1,600', '1,700', '1,750', '1,800', '1,800-', '1,815', '1,900', '1-2', '1-3', '1-4', '1-5', '1-800-', '1-800-232-0233', '1-800-273-8255', '1-800-318-2596', '1-844-330-2020', '1-877-229-8493', '1.1', '1.14', '1.15', '1.2', '1.25', '1.26', '1.3', '1.35', '1.37', '1.4', '1.45', '1.47', '1.5', '1.6', '1.68', '1.7', '1.75', '1.8', '1.85', '1.9', '1/1', '1/10', '1/15', '1/2', '1/20', '1/24', '1/3', '1/31', '1/4', '1/5', '1/6', '1/7', '1/8', '10', '10,000', '10,000+', '10,200', '10-0', '10-11', '10-15', '10-2', '10-4', '10.1', '10.10', '10.2', '10.3', '10.4', '10.5', '10.6', '10.7', '10.8', '10.9', '10/10', '10/11', '10/12', '10/14', '10/15', '10/17', '10/19', '10/21', '10/22', '10/24', '10/25', '10/26', '10/3', '10/30', '10/31', '10/8', '100', '100,00', '100,000', '100,000+', '100,441'

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#### SVM

In [ ]:
#%%time
#from sklearn import svm

#svm_lineal_model = svm.SVC(kernel= "linear", C = 1.0)
#svm_lineal_model.fit(X=tfidf_train, y= y_train)

In [ ]:
!ls